## The purpose of this notebook is to prep the raw data for citation network analyses

## Packages and Modules

In [1]:
# Data Manipulation
import pandas as pd 
import numpy as np 

# Data visualization
import matplotlib.pyplot as plt 
import seaborn as sns 

# NLP
import nltk 
nltk.data.path.append("../models/")
from nltk.util import ngrams  
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import re 

# Preferences
from IPython.display import clear_output

## The dataset

In [2]:
raw_data = pd.read_csv("../data/02_intermediate/newlines_apos_removed_paragraphs_added.csv")
data = raw_data.copy()
data.sample(5)

,article_title,authors,abstract,full_text,citation_info,processed,paragraph
3092,Group instruction with profoundly retarded per...,"M M Schepis, D H Reid, and J R Fitzgerald",We evaluated a group instruction program for t...,"JOURNAL OF APPLIED BEHAVIOR ANALYSIS\n\n1987, ...",J Appl Behav Anal. 1987 Spring; 20(1): 97–105.,"JOURNAL OF APPLIED BEHAVIOR ANALYSIS 1987, 20,...","['JOURNAL OF APPLIED BEHAVIOR ANALYSIS', '1987..."
4393,The Journal of The Experimental Analysis of Be...,Victor G Laties,The Journal of the Experimental Analysis of Be...,JOURNAL OF THE EXPERIMENTAL ANALYSIS OF BEHAVI...,J Exp Anal Behav. 2008 Jan; 89(1): 95–109.,JOURNAL OF THE EXPERIMENTAL ANALYSIS OF BEHAVI...,['JOURNAL OF THE EXPERIMENTAL ANALYSIS OF BEHA...
6920,Schedule-induced escape from fixed-interval re...,Thomas G. Brown and Randall K. Flory,Pigeons trained to peck one of two keys for fo...,JOURNAL OF THE EXPERIMENTAL ANALYSIS OF BEHAVI...,J Exp Anal Behav. 1972 May; 17(3): 395–403.,JOURNAL OF THE EXPERIMENTAL ANALYSIS OF BEHAVI...,['JOURNAL OF THE EXPERIMENTAL ANALYSIS OF BEHA...
2432,Prompted voiding to reduce incontinence in com...,V K Adkins and R M Mathews,In-home caregivers were taught to implement a ...,(cid:74)(cid:79)(cid:85)(cid:82)(cid:78)(cid:6...,J Appl Behav Anal. 1997 Spring; 30(1): 153–156.,(cid:74)(cid:79)(cid:85)(cid:82)(cid:78)(cid:6...,['(cid:74)(cid:79)(cid:85)(cid:82)(cid:78)(cid...
3621,Pyramid training of elementary school teachers...,"Fredric H. Jones, William Fremouw,2 and Steve...",NaN,"JOURNAL OF APPLIED BEHAVIOR ANALYSIS\n\n1977, ...",J Appl Behav Anal. 1977 Summer; 10(2): 239–253.,"JOURNAL OF APPLIED BEHAVIOR ANALYSIS 1977, 10,...","['JOURNAL OF APPLIED BEHAVIOR ANALYSIS', '1977..."


In [3]:
# Isolate the journal names
journal_names = []
for citation in data['citation_info'].astype(str):
    # Find the year via regex
    match = re.search(r'\d{4}', citation)
    if match:
        end_index = match.start()
        journal_name = citation[:end_index].strip()
    else:
        journal_name = citation  # If no year is found, assume the whole string is the journal name

    journal_names.append(journal_name)

data['journal'] = journal_names
data['journal'].unique()

array(['Anal Verbal Behav.', 'nan', 'VB News.', 'Behav Anal Pract.',
       'J Appl Behav Anal.', 'J Exp Anal Behav.', 'Perspect Behav Sci.',
       'Behav Anal.'], dtype=object)

In [4]:
# Swap out the journal shorthand with an easier to use abbreviation
journal_dict = {
    'Anal Verbal Behav.': 'TAVB', 
    'VB News.': 'TAVB', 
    'Behav Anal Pract.': 'BAP', 
    'J Appl Behav Anal.': 'JABA', 
    'J Exp Anal Behav.': 'JEAB', 
    'Perspect Behav Sci.': 'PoBS', 
    'Behav Anal': 'PoBS'
}
data['journal'] = data['journal'].map(journal_dict)
data['journal'].unique()

array(['TAVB', nan, 'BAP', 'JABA', 'JEAB', 'PoBS'], dtype=object)

In [5]:
# Replace "- "
data['processed'] = [str(val).replace("- ", "") for val in data['processed']]

In [6]:
# Function for separating the manuscript into the respective sections
def section_isolator(processing_col, section_start_name, next_section_name):
        new_col = []
        for val in processing_col:
            try:
                new_col.append(str(val).lower().split(f"{section_start_name}")[1])
            except:
                new_col.append(val)
        new_col = [str(val).split(f"{next_section_name}")[0] for val in new_col]
        return new_col

In [7]:
# Function to reverse a string
def reverse_string(s):
    return s[::-1]

# Function to find the last part of text after the last occurrence of 'reference'
def get_references_section(text):
    # Reverse the text and the word 'references'
    reversed_text = reverse_string(text.lower())
    reversed_word = reverse_string(" references")

    # Find the index of the last occurrence of 'references'
    idx = reversed_text.find(reversed_word)

    # If 'references' is found, return the part after it; otherwise, return the original text
    if idx != -1:
        # Calculate the actual index in the original text
        actual_idx = len(text) - idx - len("references")
        return text[actual_idx:].strip()
    else:
        return text

In [8]:
# Isolate the references section
new_col = []
for val in data['processed']:
    references_section = get_references_section(val)
    new_col.append(references_section)

data['references'] = new_col
data['references'] = data['references'].astype(str)

## Create a matrix of the count of each article being cited within the others

In [9]:
# Lowercase the titles and references once
data['article_title'] = data['article_title'].str.lower()
data['references'] = data['references'].str.lower()

# Create a dictionary to hold the data
cite_data = {}

# Get unique titles
unique_titles = data['article_title'].unique()
total_titles = len(unique_titles)

# Loop through unique titles
for index, title in enumerate(unique_titles, 1):
    # Escape any special characters in the title for regular expression
    escaped_title = re.escape(title)

    # Use vectorized string contains method with the escaped title
    cite_data[title] = data['references'].str.contains(escaped_title, regex=True).astype(int)

    # Print progress update every 50 articles
    if index % 50 == 0 or index == total_titles:
        clear_output()
        print(f"Processed {index} of {total_titles} titles.")

# Convert the dictionary to a DataFrame
cite_matrix = pd.DataFrame(cite_data)

Processed 9334 of 9334 titles.


In [10]:
# Save citation matrix
cite_matrix.index = data['article_title']
cite_matrix.to_csv("../data/02_intermediate/citation_matrix_using_contains.csv")

In [11]:
# Save data file
data.to_csv("../data/02_intermediate/raw_data_cleaned_for_citation_matrix.csv")